In [22]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
import sklearn
from sklearn.preprocessing import StandardScaler
import importlib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV 
import sklearn
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier,AdaBoostClassifier
import scikitplot as skplt
import arabic_reshaper
from bidi.algorithm import get_display
from sklearn.inspection import partial_dependence
from sklearn.inspection import plot_partial_dependence
from pdpbox import pdp, info_plots
from pdpbox import pdp, get_dataset, info_plots
from pdpbox import pdp_plot_utils
from pptx import Presentation
from io import StringIO
from io import BytesIO
from pptx.util import Inches

### Read, Warngle and Tranform Data for ML Model

In [6]:
#first read#
data=pd.read_csv('raw_2500.csv',encoding='utf-8-sig')
#data.rename(columns=dict(zip(data.columns.tolist(),['feat'+i for i in np.arange(1,len(data.columns)+1).astype(str)])),inplace=True)
#date#
tarikh=pd.read_csv('183104-arash.csv',encoding='utf-8-sig')
date=tarikh.iloc[:,[2,3,5,6,7,8,9,10,20]]
date['VAR3']=date['VAR3'].astype('datetime64[ns]').dt.date
date.drop_duplicates(inplace=True)
#x_vars=pd.read_csv('selected_varbs1.csv',encoding='utf-8-sig')
x_vars=pd.read_csv('selected_varbs2.csv',encoding='utf-8-sig')

data['dep_var']=1*data.iloc[:,22]+2*data.iloc[:,21]+3*data.iloc[:,23]
df=data.loc[:,x_vars.definition]
df['Start_GDate/Time']=data['Start_GDate/Time']
df['dep_var'] = data['dep_var']
df.dropna(inplace=True)
#merge df and date#
df['Start_GDate/Time']=df['Start_GDate/Time'].astype('datetime64[ns]').dt.date
df=df.merge(date,how='left',left_on='Start_GDate/Time',right_on='VAR3')
df.dropna(inplace=True)
df=df.loc[df['dep_var']!=0]

qwe=['HourlyTrafficDataset_TotalCarFlowRates(Veh/min)_mean',
     'HourlyTrafficDataset_HeadWay(sec)_mean','HourlyTrafficDataset_TotalAverageSpeed(km/h)_mean']
ffr=['Flowrate','Headway','Speed']

for i,j in zip(qwe,ffr):
    bins = [0*df[i].max(), 0.2*df[i].max(), 0.4*df[i].max(), 0.6*df[i].max(), 0.8*df[i].max(), 1*df[i].max()]
    group_names = [1,2,3,4,5]
    
    df[j]=pd.cut(df[i], bins, labels=group_names)
    
trf_cor=[]
for i in ffr:
    s=abs(pd.concat([df[['dep_var']],pd.get_dummies(df[i],prefix=i+'_')]
                    ,axis=1).corr()[['dep_var']]).sort_values(by='dep_var',ascending=False).index[1]   
    trf_cor.append(s)
    df=pd.concat([df,pd.get_dummies(df[i],prefix=i+'_')],axis=1)
#create dummies of calendar varbs 
yu={}
for t in tarikh.iloc[:,[5,6,7,8,9,10]].columns:
    if t=="VAR11":
        yu[t]=pd.get_dummies(df[t],drop_first=True).add_suffix('th year')
    else:
        yu[t]=pd.get_dummies(df[t],drop_first=True) 
added=pd.concat(yu,axis=1)
added.columns=[added.columns[i][1] for i in range(len(added.columns))]
df=pd.concat([df,added],axis=1)

df['mot_cycl']=np.where((df.loc[:,[x for x in df.columns if "موتورسیکلت" in x]].sum(axis=1))>0,1,0)
df['car']=np.where((df.loc[:,[x for x in df.columns if "سواری" in x]].sum(axis=1))>0,1,0)

y = df['dep_var']
c=abs(df.corr()[['dep_var']]).sort_values(by='dep_var',ascending=False).head(n=34).index.tolist()
c.remove('dep_var')
c.remove('فوریه')
c.remove('VAR11')
c.remove('4.0th year')
#c.remove('CrashDataset_Lighting_Condition_شب بدون روشنایی کافی')
#c.remove('CrashDataset_Vehicle_1_Group_اتوبوس')
#c.remove('CrashDataset_Roadway_Surface_Condition_یخبندان و برفی')

#c.remove('رجب')

#c.remove('CrashDataset_Vehicle_12_Group_سواری')
#c.remove('CrashDataset_Vehicle_12_Group_احشام')
c.remove('CrashDataset_Vehicle_12_Group_موتورسیکلت')
c.remove('CrashDataset_Vehicle_1_Group_موتورسیکلت')
#c.remove('CrashDataset_Vehicle_1_Group_سواری')
ffr=['Flowrate','Headway','Speed']

c=[x for x in c if "HourlyTrafficDataset" not in x]
ert=c+qwe
ert.remove('Speed__1')
x = df.loc[:,ert]
#names=pd.read_csv('check.csv',encoding='utf-8-sig')
#x.columns=names['def2'][:-1]
#x.drop(columns=['CrashDataset_Vehicle_1_Action_حرکت به جلو'],inplace=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=8)

### Create a GridSearch for algorithms who are exist in SKLearn

In [ ]:
# import sklearn
#ss=['CrashDataset_Vehicle_1_Action_سبقت','CrashDataset_Vehicle_1_Action_گردش به چپ']
#param_grid = {'n_estimators': [15,35,70,141,210],
#              'max_depth':np.arange(5,25,2),
#              'max_features':('auto','log2'),
#              'min_samples_split': (np.arange(0.01,0.21,0.02)*len(x_train)).astype(int)}
param_grid = {'n_estimators': list(np.arange(1,500,4).astype(int)),
             'algorithm':['SAMME', 'SAMME.R']} #param for adaboost

grid = GridSearchCV(AdaBoostClassifier(),param_grid, cv=5, verbose=50,refit = True,n_jobs=-1) 
grid.fit(x_train, y_train)

In [ ]:

#ERT=ExtraTreesClassifier(max_depth= 13,
#                         max_features= 'auto',
#                               min_samples_split= 11,
#                               n_estimators= 141)

#ERT=ExtraTreesClassifier(max_depth= 9,
# max_features= 'auto',
# min_samples_split= 79,
# n_estimators= 35)
#ERT.fit(x_train,y_train)
#y_train_pred = ERT.predict(x_train)
#y_test_pred = ERT.predict(x_test)


ADAB=AdaBoostClassifier(n_estimators= 73,algorithm='SAMME')
ADAB.fit(x_train,y_train)
y_train_pred = ADAB.predict(x_train)
y_test_pred = ADAB.predict(x_test)

print('accuracy train:  ' + str(sklearn.metrics.accuracy_score(y_train, y_train_pred)))
print('cf_matrix on train:    ' + str(sklearn.metrics.confusion_matrix(y_train, y_train_pred)))

print('accuracy test:   ' + str(sklearn.metrics.accuracy_score(y_test, y_test_pred)))
print('cf_matrix: on test:     ' + str(sklearn.metrics.confusion_matrix(y_test, y_test_pred)))

print('precision_score train:  ' + str(sklearn.metrics.precision_score(y_train, y_train_pred, average="weighted")))
print('precision_score test:   ' + str(sklearn.metrics.precision_score(y_test, y_test_pred, average="weighted")))

print('recall_score train:  ' + str(sklearn.metrics.recall_score(y_train, y_train_pred, average="weighted")))
print('recall_score test:   ' + str(sklearn.metrics.recall_score(y_test, y_test_pred, average="weighted")))

print('f1_score train:  ' + str(sklearn.metrics.f1_score(y_train, y_train_pred, average="weighted")))
print('f1_score test:   ' + str(sklearn.metrics.f1_score(y_test, y_test_pred, average="weighted")))

### Grid Search for XGBoost

In [ ]:

dtrain = xgb.DMatrix(x_train, label=np.array(y_train))
dtest = xgb.DMatrix(x_test, label=np.array(y_test))
param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
xgb_m = XGBClassifier()
grid = GridSearchCV(xgb_m,param_grid, cv=5, verbose=50,refit = True,n_jobs=-1) 
grid.fit(x_train, y_train)

In [ ]:
grid.best_params_
bst =  XGBClassifier(colsample_bytree= 0.6,
                     gamma= 5,
                     max_depth= 3,
                     min_child_weight= 1,
                     subsample= 1.0)
bst.fit(x_train,y_train)
y_train_pred = bst.predict(x_train)
y_test_pred = bst.predict(x_test)

print('accuracy train:  ' + str(sklearn.metrics.accuracy_score(y_train, y_train_pred)))
print('cf_matrix on train:    ' + str(sklearn.metrics.confusion_matrix(y_train, y_train_pred)))

print('accuracy test:   ' + str(sklearn.metrics.accuracy_score(y_test, y_test_pred)))
print('cf_matrix: on test:     ' + str(sklearn.metrics.confusion_matrix(y_test, y_test_pred)))

print('precision_score train:  ' + str(sklearn.metrics.precision_score(y_train, y_train_pred, average="weighted")))
print('precision_score test:   ' + str(sklearn.metrics.precision_score(y_test, y_test_pred, average="weighted")))

print('recall_score train:  ' + str(sklearn.metrics.recall_score(y_train, y_train_pred, average="weighted")))
print('recall_score test:   ' + str(sklearn.metrics.recall_score(y_test, y_test_pred, average="weighted")))

print('f1_score train:  ' + str(sklearn.metrics.f1_score(y_train, y_train_pred, average="weighted")))
print('f1_score test:   ' + str(sklearn.metrics.f1_score(y_test, y_test_pred, average="weighted")))

### Plotting ROC Curve for ERT

In [ ]:

fig, ax = plt.subplots(figsize=(12,5))

skplt.metrics.plot_roc_curve(y_test, ERT.predict_proba(x_test),ax=ax)
#plt.ylim(0, 0.5)
#gradient_image(ax, direction=0, extent=(0, 1, 0, 1), transform=ax.transAxes,cmap=plt.cm.Greens, cmap_range=(0.2, 0))

    
plt.ylabel('True Positive Rate',fontname='Times New Roman',fontweight="bold")
plt.xlabel('False Positive Rate',fontname='Times New Roman',fontweight="bold")
plt.xticks(fontname='Times New Roman')
plt.yticks(fontname='Times New Roman')
L = ax.legend()
plt.setp(L.texts, family='Times New Roman')
#plt.legend(['Accuracy'])
plt.title('ROC Curves',fontname='Times New Roman')

plt.grid(True,which='major',axis='y')
plt.savefig('figs/ROC Curves.png',dpi=350,bbox_inches='tight')

### Plotting Feature Importance

In [13]:
gbr=ERT
check=pd.read_csv('check2.csv',encoding='utf-8-sig')

cols_df=pd.DataFrame(x_train.columns,columns=['varb'])
cols_df['def']=check['def']
cols_df.dropna(inplace=True)
cols_df['importances_gbr'] = ERT.feature_importances_

z = [ ]

for item in [t for t in x_train.columns if t!='y']:
    z.append(get_display(arabic_reshaper.reshape(item)))
cols_df['fd']=z
cols_df['hhh']=['feat'+i for i in np.arange(1,len([t for t in x_train.columns if t!='y'])+1).astype(str)]
tbp=cols_df.sort_values(by=['importances_gbr'],ascending=False).head(n=10).reset_index(drop=True)
plt.rcParams["font.family"] = "Times New Roman"

fig, ax = plt.subplots(figsize=(10,10))
graph=sn.barplot(x="importances_gbr", y="def", data=cols_df.sort_values('importances_gbr', ascending=False), palette="viridis")
for  p in ax.patches:
    width = p.get_width()
    ax.text(width +0.01,
            p.get_y()+p.get_height()/2. + 0.2,
            '{:1.4f}'.format(width),
            ha="center")
plt.title(get_display(arabic_reshaper.reshape('Importance of Features')))
plt.ylabel('Feature',fontname='Times New Roman',fontweight="bold", fontsize=12)
plt.xlabel('Importance (0-1)',fontname='Times New Roman',fontweight="bold", fontsize=12)
#plt.savefig('feature_importance.png',dpi=150,bbox_inches='tight')

# sklearn pdps

In [18]:
cols_df.loc[1,'def']='motorcycle involved'

In [ ]:

for i in np.arange(0,len(x_train.columns),1):
    
    fig, ax1 = plt.subplots(figsize=(6, 4))
    feat_id=i
    yy1=partial_dependence(ERT, x_train, features=feat_id)[0][0] #class
    xx=partial_dependence(ERT, x_train, features=feat_id)[1][0] 
    ax1.plot(xx,yy1,label='PDO',linestyle='-.',linewidth=2)

    yy2=partial_dependence(ERT, x_train, features=feat_id)[0][1] #class
    ax1.plot(xx,yy2,label='Injury',linestyle='--',linewidth=2)

    yy3=partial_dependence(ERT, x_train, features=feat_id)[0][2] #class
    kamine=min([min(yy1),min(yy2),min(yy3)])
    bishine=max([max(yy1),max(yy2),max(yy3)])+0.1

    onvan=['PDO: '+'min='+str(round(min(yy1),3))+', max='+str(round(max(yy1),3)),
                       'Injury: '+'min='+str(round(min(yy2),3))+', max='+str(round(max(yy2),3)),
                       'Fatal: '+'min='+str(round(min(yy3),3))+', max='+str(round(max(yy3),3))] 

    ax1.plot(xx,yy3,label='Fatal',linestyle='dotted',linewidth=2)
    ax1.legend(labels=onvan,fontsize=6,loc='best')
    ax1.set_xlabel(xlabel=cols_df['def'][feat_id])
    ax1.set_ylabel(ylabel="Probability")
    

    plt.savefig('figs/pdp/'+cols_df['def'][feat_id].split('(')[0]+'.png',dpi=350,bbox_inches='tight')

# Ice plots

In [ ]:

#PDP Plot
#ICE Plot
#fig, pdp_ax = plt.subplots(figsize=(6,5))
x_train.columns=cols_df['def']
for s,q in zip([0,1,2],['PDO class','Injury class','Fatal class']):
    
    for  ix,label in (zip(cols_df['varb'],cols_df['def'])):
        pdp_age = pdp.pdp_isolate(model=ERT, dataset=x_train, model_features=x_train.columns, feature=label)
        fig, axes=pdp.pdp_plot(pdp_age, label, plot_lines=True, center=True,which_classes=[s],
                              plot_params = {
                    'title': 'ICE for feature "%s"' % label,'title_fontsize': 10,
                    'subtitle_fontsize': 8,
                    'font_family': 'Times New Roman','line_cmap': 'Blues'})
        fig = matplotlib.pyplot.gcf()
        fig.set_size_inches(6, 5)
        plt.xlabel(q)
        plt.ylabel('Change in Probability')
        plt.tight_layout()
        plt.savefig('figs/ice/centered/'+label.split('(')[0]+'_'+q+'_centered.png',dpi=350,bbox_inches='tight')
#fig, ax = plt.subplots(figsize=(12,5))

### Interaction Partial Dependance Plot

In [31]:


l=cols_df.sort_values(by='importances_gbr',ascending=False).head(n=4)['def']
m=cols_df.tail(n=3)['def']
pairs=[(x,y)for x in l for y in m]

p=cols_df.sort_values(by='importances_gbr',ascending=False).head(n=4)['varb']
q=cols_df.tail(n=3)['varb']
feds=[(x,y)for x in p for y in q]

# Override to fix matplotlib issue
def _pdp_contour_plot_override(X, Y, pdp_mx, inter_ax, cmap, norm, inter_fill_alpha, fontsize, plot_params):
    contour_color = plot_params.get('contour_color', 'white')
    level = np.min([X.shape[0], X.shape[1]])
    c1 = inter_ax.contourf(X, Y, pdp_mx, N=level, origin='lower', cmap=cmap, norm=norm, alpha=inter_fill_alpha)
    c2 = inter_ax.contour(c1, levels=c1.levels, colors=contour_color, origin='lower')
    inter_ax.clabel(c2, fontsize=fontsize, inline=1)
    inter_ax.set_aspect('auto')
    return c1
#prs = Presentation()
pdp_plot_utils._pdp_contour_plot = _pdp_contour_plot_override
interact_pdps={}
for pair, fed in zip(pairs,feds):

    pdp_inter = pdp.pdp_interact(model=gbr, dataset=x_train, model_features=x_train.columns, features=pair)
    interact_pdps[pair]=pdp_inter

In [ ]:
classes=dict(zip([0,1,2],['PDO','injury','fatal']))

for pair, fed in zip(pairs,feds):
    for rade in [0,1,2]:
        df_pdp=interact_pdps[pair][rade].pdp
        zz=df_pdp.iloc[:,2]
        xx=df_pdp.iloc[:,0]
        yy=df_pdp.iloc[:,1]
        fig = go.Figure(data =go.Contour(z=zz, x=xx, y=yy,colorscale='viridis', contours=dict(coloring ='heatmap',showlabels = True)))

        fig.update_layout(autosize=True,width=600,height=800,margin=dict(l=20,r=20,b=20,t=20,pad=20),)
        fig.update_xaxes(title=pair[0],title_font_family="Times New Roman",title_font_size=20)
        fig.update_yaxes(title=pair[1],title_font_family="Times New Roman",title_font_size=20)
        fig.update_layout(title={'text': "<b>Interaction PDP:"+classes[rade]+'_class','y':1,'x':0.5,'xanchor': 'center'
                                 ,'yanchor': 'top','font_family':"Times New Roman",'font_size':24})
        fig.write_image("figs/interactive_pdp/"+classes[rade]+'_'+pair[0]+'_'+pair[1].split('(')[0]+".png")
#"interactive_pdp/pdpinteract_"+pairs[0][0]+'_'+pairs[0][1]+